## Requirements
It is expected that simulator can perform following:

- initialize state
- read program, and for each gate:
    - calculate matrix operator
    - apply operator (modify state)

- perform multi-shot measurement of all qubits using weighted random technique

It is up to you how you will organize code, but this is our suggestion:

### Input format (quantum program)

It is enough if simulator takes program in following format:

```
[
  { \"unitary\": [[0.70710678, 0.70710678], [0.70710678, -0.70710678]], \"target\": [0] },
  { \"unitary\": [ [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0] ], \"target\": [0, 1] }
  ...
]
```

Or, you can define unitaries in the code, and accept program with gate names instead:

```
[
  { \"gate\": \"h\", \"target\": [0] },
  { \"gate\": \"cx\", \"target\": [0, 1] }
  ...
]
```

In [1]:
import numpy as np
import unittest
from numpy.random import multinomial
import math
import json
import functools

ModuleNotFoundError: No module named 'numpy'

# Helper functions

In [2]:
def ln2(number):
    return math.frexp(number)[1] - 1

In [3]:
class TestHelpers(unittest.TestCase):

    def test_ln2_2(self):
        self.assertEqual(ln2(2), 1)

    def test_ln2_8(self):
        self.assertEqual(ln2(8), 3)
    def test_ln_2(self):
        self.assertEqual(ln2(256), 8)

suite = unittest.TestLoader().loadTestsFromTestCase(TestHelpers)
unittest.TextTestRunner().run(suite)

...
----------------------------------------------------------------------
Ran 3 tests in 0.003s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

# Get ground state [DONE]

In [4]:
def get_ground_state(num_qubits):
    # return vector of size 2**num_qubits with all zeroes except first element which is 1

#     assert num_qubits > 0
    
    ground_state = a = np.zeros(2**num_qubits)
    ground_state[0] = 1
    return ground_state

In [5]:
class TestGetGroundState(unittest.TestCase):

#     def test_invalid_input(self):
#         self.assertRaises(get_ground_state(0), AssertionError)

    def test_single_qubit(self):
        self.assertSequenceEqual(
            list(get_ground_state(1)),
            list(np.array([1, 0]))
        )

    def test_multi_qubit(self):
        self.assertSequenceEqual(
            list(get_ground_state(3)),
            list(np.array([1, 0, 0, 0, 0 ,0, 0, 0]))
        )

suite = unittest.TestLoader().loadTestsFromTestCase(TestGetGroundState)
unittest.TextTestRunner().run(suite)

..
----------------------------------------------------------------------
Ran 2 tests in 0.002s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

# Get Operator [not started]

In [6]:
# Define 2x2 Identity
I = np.identity(2)

# Define X gate
X = np.array([
[0, 1],
[1, 0]
])

# Define Y gate
Y = np.array([
[0, 0-1j],
[0+1j, 0]
])

# Define Z gate
Z = np.array([
[1, 0],
[0, -1]
])

# Define H gate
H = np.array([
[1, 1],
[1, -1]
])/math.sqrt(2)

# Define parametric gate u3
U3 = np.array([
["cos(theta/2)", "-exp(i * lambda_angle) * sin(theta / 2)"],
["exp(i * phi) * sin(theta / 2)", "exp(i * lambda_angle + i * phi) * cos(theta / 2)"]    
])

# Define projection operator |0><0|
P0x0 = np.array([
[1, 0],
[0, 0]
])

# Define projection operator |1><1|
P1x1 = np.array([
[0, 0],
[0, 1]
])

KNOWN_GATES = {
    "I": I,
    "X": X,
    "Y": Y,
    "Z": Z,
    "H": H,
    "U3": U3,
    "P0": P0x0,
    "P1": P1x1
}

In [75]:
def get_operator(total_qubits, gate_unitary, target_qubits, params = None):
    # return unitary operator of size 2**n x 2**n for given gate and target qubits

    CONTOL_GATE_INDICATOR = 'C'
    
    def get_gate_from_name(name, params=None):
        PARAMETRIC_GATE_NAME = "U3"
        upper_case_name = name.upper()

        print(f'get_single_qubit_gate({name}, {params}=None)')
        if name == PARAMETRIC_GATE_NAME and params is not None:
            theta = params.get('theta')
            phi = params.get('phi')
            lambda_angle = params.get('lambda')
            print(f'This will break! {theta}, {phi}, {lambda_angle}')
            parametric_gate = KNOWN_GATES[PARAMETRIC_GATE_NAME]
            for entry in parametri
            return parametric_gate
        return KNOWN_GATES[name]
    
    def multiply_gates(gate_unitary, target_qubits, projector=None, control_qubit=None):
        gates = [I] * total_qubits
        for target_qubit in target_qubits:
            gates[target_qubit] = gate_unitary
        if projector is not None and control_qubit is not None:
            gates[control_qubit] = projector
        return functools.reduce(lambda a,b : np.kron(a, b), gates)
    
    is_a_controlled_gate = gate_unitary.upper()[0] == CONTOL_GATE_INDICATOR
    if is_a_controlled_gate:
        gate = get_gate_from_name(gate_unitary[1:], params)
    else:
        gate = get_gate_from_name(gate_unitary, params)

    if is_a_controlled_gate:
        result \
        = multiply_gates(I, [target_qubits[1]], P0x0, target_qubits[0]) \
        + multiply_gates(gate, [target_qubits[1]], P1x1, target_qubits[0])
#         print('controlled-gate:\n', np.array2string(result))
        return result
    else:
        result = multiply_gates(gate, target_qubits)
#         print('gate:\n', np.array2string(result))
        return result
    
    return gate_unitary

In [76]:
class TestGetOperator(unittest.TestCase):

    def test_n_is_1_X_on_0(self):
        np.testing.assert_array_equal(
            get_operator(1, "X", [0]),
            np.array([
                [0,1],
                [1,0]
            ])
        )

    def test_n_is_2_X_on_0(self):
        np.testing.assert_array_equal(
            get_operator(2, "X", [0]),
            np.array([
                [0,0,1,0],
                [0,0,0,1],
                [1,0,0,0],
                [0,1,0,0]
            ])
        )
    
    def test_n_is_2_X_on_1(self):
        np.testing.assert_array_equal(
            get_operator(2, "X", [1]),
            np.array([
                [0,1,0,0],
                [1,0,0,0],
                [0,0,0,1],
                [0,0,1,0]
            ])
        )

    def test_n_is_2_H_on_0_and_1(self):
        np.testing.assert_allclose(
            get_operator(2, "H", [0,1]),
            np.array([
                [1,1,1,1],
                [1,-1,1,-1],
                [1,1,-1,-1],
                [1,-1,-1,1]
            ])/2
            , rtol=1e-5, atol=0
        )

    def test_n_is_3_X_on_2(self):
        np.testing.assert_array_equal(
            get_operator(3, "X", [2]),
            np.array([
                [0, 1, 0, 0, 0, 0, 0, 0,],
                [1, 0, 0, 0, 0, 0, 0, 0,],
                [0, 0, 0, 1, 0, 0, 0, 0,],
                [0, 0, 1, 0, 0, 0, 0, 0,],
                [0, 0, 0, 0, 0, 1, 0, 0,],
                [0, 0, 0, 0, 1, 0, 0, 0,],
                [0, 0, 0, 0, 0, 0, 0, 1,],
                [0, 0, 0, 0, 0, 0, 1, 0,]
            ])
        )

    def test_n_is_3_CNOT_on_0_and_2(self):
        np.testing.assert_array_equal(
            get_operator(3, "CX", [0,2]),
            np.array([
                [1, 0, 0, 0, 0, 0, 0, 0],
                [0, 1, 0, 0, 0, 0, 0, 0],
                [0, 0, 1, 0, 0, 0, 0, 0],
                [0, 0, 0, 1, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 1, 0, 0],
                [0, 0, 0, 0, 1, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 1],
                [0, 0, 0, 0, 0, 0, 1, 0]
            ])
        )

    def test_n_is_1_U3_on_0(self):
        np.testing.assert_array_equal(
            get_operator(1, "U3", [0], { "theta": 3.1415, "phi": 1.5708, "lambda": -3.1415 }),
            np.array([
                [0,1],
                [1,0]
            ])
        )

suite = unittest.TestLoader().loadTestsFromTestCase(TestGetOperator)
unittest.TextTestRunner().run(suite)

/opt/conda/lib/python3.8/site-packages/numpy/testing/_private/utils.py:788: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  val = comparison(x, y)
F......

get_single_qubit_gate(U3, {'theta': 3.1415, 'phi': 1.5708, 'lambda': -3.1415}=None)
This will break! 3.1415, 1.5708, -3.1415
get_single_qubit_gate(X, None=None)
get_single_qubit_gate(H, None=None)
get_single_qubit_gate(X, None=None)
get_single_qubit_gate(X, None=None)
get_single_qubit_gate(X, None=None)
get_single_qubit_gate(X, None=None)



FAIL: test_n_is_1_U3_on_0 (__main__.TestGetOperator)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-76-194b9fc3506c>", line 77, in test_n_is_1_U3_on_0
    np.testing.assert_array_equal(
  File "/opt/conda/lib/python3.8/site-packages/numpy/testing/_private/utils.py", line 930, in assert_array_equal
    assert_array_compare(operator.__eq__, x, y, err_msg=err_msg,
  File "/opt/conda/lib/python3.8/site-packages/numpy/testing/_private/utils.py", line 840, in assert_array_compare
    raise AssertionError(msg)
AssertionError: 
Arrays are not equal

Mismatched elements: 1 / 1 (100%)
 x: array([['cos(theta/2)', '-exp(i * lambda) * sin(theta / 2)'],
       ['exp(i * phi) * sin(theta / 2)',
        'exp(i * lambda + i * phi) * cos(theta / 2)']], dtype='<U42')
 y: array([[0, 1],
       [1, 0]])

----------------------------------------------------------------------
Ran 7 tests in 0.057s

FAILED (failures=1)


<unittest.runner.TextTestResult run=7 errors=0 failures=1>

# Run Program [not started]


In order to implement simulator, it is best if you have function which returns operator for any unitary targeting any qubit(s) for any circuit size, something like:

```get_operator(total_qubits, gate_unitary, target_qubits)```

But this is not trivial so it is enough if you can implement it for any 1-qubit gates and CNOT only.

If you are still enthusiastic and you wish to implement universal operator function then please refer to:

- [qosf-simulator-task-additional-info.pdf|https://github.com/quantastica/qosf-mentorship/blob/master/qosf-simulator-task-additional-info.pdf]

- Book Nielsen, Michael A.; Chuang, Isaac (2000). Quantum Computation and Quantum Information, 10th Anniversary Edition, Section 8.2.3, Operator-Sum Representation




In [48]:
def run_program(initial_state, program):
    # read program, and for each gate:
    #   - calculate matrix operator
    #   - multiply state with operator
    # return final state
    
    n = ln2(len(initial_state))
    state = initial_state
#     print('initial state: ', state)

    for gate_operation in program:
        params = gate_operation.get("params")
#         print(f'params: {params}')
        gate = gate_operation.get("gate")
#         print('gate: ', gate)

        target = gate_operation.get("target")
#         print('target: ', target)

        matrix_operator = get_operator(n, gate, target, params)
#         print('matrix: ', matrix_operator)
        
        state = np.dot(matrix_operator, state)
#         print('state: ', state)

        
#     print('final state: ', state)
    return state

In [49]:
class TestRunProgram(unittest.TestCase):

    def test_run_program_bell_00(self):
        np.testing.assert_allclose(
            run_program(
                np.array([1, 0, 0, 0]),
                [
                    { "gate": "h", "target": [0] },
                    { "gate": "cx", "target": [0, 1] }
                ]
            ),
            np.array([0.70710678, 0, 0, 0.70710678]),
            rtol=1e-5, atol=0
        )

    def test_run_program_uniform_superposition(self):
        np.testing.assert_allclose(
            run_program(
                np.array([1, 0, 0, 0]),
                [
                    { "gate": "h", "target": [0,1] }
                ]
            ),
            [0.5]*4,
            rtol=1e-5, atol=0
        )

    def test_run_program_parametric_gate(self):
        np.testing.assert_allclose(
            run_program(
                np.array([1, 0]),
                [
                    { "gate": "u3", "params": { "theta": 3.1415, "phi": 1.5708, "lambda": -3.1415 }, "target": [0] }
                ]
            ),
            np.array([ 0+0j, 0+1j]),
            rtol=1e-5, atol=0
        )

suite = unittest.TestLoader().loadTestsFromTestCase(TestRunProgram)
unittest.TextTestRunner().run(suite)

.E.
ERROR: test_run_program_parametric_gate (__main__.TestRunProgram)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-49-66b4f895197b>", line 30, in test_run_program_parametric_gate
    run_program(
  File "<ipython-input-48-6b8b845677c8>", line 23, in run_program
    state = np.dot(matrix_operator, state)
  File "<__array_function__ internals>", line 5, in dot
ValueError: data type must provide an itemsize

----------------------------------------------------------------------
Ran 3 tests in 0.112s

FAILED (errors=1)


<unittest.runner.TextTestResult run=3 errors=1 failures=0>

# Measure All [DONE; remove comments]

In [11]:
def measure_all(state_vector):
    # choose element from state_vector using weighted random and return it's index
    def get_probability(amplitude):
        pa = np.abs(amplitude)**2
        return pa
    
    probabilities = np.fromiter((get_probability(amplitude) for amplitude in state_vector), state_vector.dtype) 
#     print('prob', probabilities)
    
    result_array = multinomial(1, probabilities)
#     print('results', result_array)    
    indices = np.where(result_array == 1)
#     print('indices: ', indices)    
    index = indices[0][0]
#     print('index: ', index)
    
    bin_index = bin(index)[2:].zfill(ln2(len(state_vector)))
#     print('bin index: ', bin_index)
    
    return bin_index

In [12]:
class TestMeasureAll(unittest.TestCase):

    def test_measure_bell_00(self):
        actualResult = measure_all(np.array([0.70710678, 0, 0, -0.70710678]))
        self.assertTrue(
           (actualResult == '00') or (actualResult == '11')
        )

suite = unittest.TestLoader().loadTestsFromTestCase(TestMeasureAll)
unittest.TextTestRunner().run(suite)

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

# Get Counts [remove comments + fix test]

In [13]:
def get_counts(state_vector, num_shots):
    # simply execute measure_all in a loop num_shots times and
    # return object with statistics in following form:
    #   {
    #      element_index: number_of_ocurrences,
    #      element_index: number_of_ocurrences,
    #      element_index: number_of_ocurrences,
    #      ...
    #   }
    # (only for elements which occoured - returned from measure_all)

    counts = {} 
    # or listofzeros = [0] * len(state_vector)
    for x in range(num_shots):
        index = measure_all(state_vector)
        counts[index] = counts.get(index, 0) + 1
        # counts[measure_all(state_vector)] += 1 
        
    sorted_counts = json.dumps(counts, sort_keys=True, indent=4)
#     print(sorted_counts)

    # time considerarions of dic vs list
    # list needs extra pass to covert index to binary
    # dic can take binary string from measure all, but does this affect performance of get_counts
    # dic->json goves output in required format
    
    return sorted_counts

In [14]:
class TestGetCounts(unittest.TestCase):

    def test_run_1000_shots(self):
        self.assertEqual(
            get_counts(np.array([0.70710678, 0, 0, -0.70710678]), 1000),
            0
        )

suite = unittest.TestLoader().loadTestsFromTestCase(TestGetCounts)
unittest.TextTestRunner().run(suite)

F
FAIL: test_run_1000_shots (__main__.TestGetCounts)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-14-84dd1cb2a53b>", line 4, in test_run_1000_shots
    self.assertEqual(
AssertionError: '{\n    "00": 501,\n    "11": 499\n}' != 0

----------------------------------------------------------------------
Ran 1 test in 0.061s

FAILED (failures=1)


<unittest.runner.TextTestResult run=1 errors=0 failures=1>

# End-to-end example

In [15]:
# Define program:

my_circuit = [
{ "gate": "h", "target": [0] }, 
{ "gate": "cx", "target": [0, 1] }
]


# Create "quantum computer" with 2 qubits (this is actually just a vector :) )

my_qpu = get_ground_state(2)


# Run circuit

final_state = run_program(my_qpu, my_circuit)


# Read results

counts = get_counts(final_state, 1000)

print(counts)

# Should print something like:
# {
#   "00": 502,
#   "11": 498
# }

# Voila!

{
    "00": 476,
    "11": 524
}


# Test execution

In [28]:
unittest.main(argv=[''], verbosity=0, exit=False)

params: None
non-parametric gate
params: None
non-parametric gate
params: {'theta': 3.1415, 'phi': 1.5708, 'lambda': -3.1415}
parametric gate
params: None
non-parametric gate


ERROR: test_run_program_parametric_gate (__main__.TestRunProgram)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-27-66b4f895197b>", line 30, in test_run_program_parametric_gate
    run_program(
  File "<ipython-input-26-e0559a0a45f7>", line 33, in run_program
    state = np.dot(matrix_operator, state)
  File "<__array_function__ internals>", line 5, in dot
ValueError: data type must provide an itemsize

FAIL: test_run_1000_shots (__main__.TestGetCounts)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-14-84dd1cb2a53b>", line 4, in test_run_1000_shots
    self.assertEqual(
AssertionError: '{\n    "00": 507,\n    "11": 493\n}' != 0

----------------------------------------------------------------------
Ran 16 tests in 0.130s

FAILED (failures=1, errors=1)
